# Curves and surfaces with differential forms

# Introduction

In this blog post, we will continue our exploration of Plateau's problem.  

We start to discuss the geometric measure theory based approach, specifically, we want to introduce a representation of curves and surfaces in 3-dimensional space using _integral currents_ and _differential forms_. This approach was created by Stephanie Wang and Albert Chen in their paper "Computing Minimal Surfaces with Differential Forms", published in 2021.

# Differential forms

Formally, a differential form $\omega$ is defined as an alternating linear functional on the tangent bundle of some manifold.   
Essentially, $\omega$ "eats" one or more tangent vectors $v_i$ of the manifold at some point and spits out a scalar number $\omega(v_1, v_2, \dots, v_n) \in \mathbb{R}$. A $k$-differential form $\omega$ "eats" $k$ tangent vectors.   
The alternating property means that for any combination of tangent vectors $v_1, v_2, \dots, v_n$ and for any $i, j \leq n$ we have $\omega(v_1, v_2, \dots, v_i, v_j, \dots, v_n) = -\omega(v_1, v_2, \dots, v_j, v_i, \dots, v_n)$. Intuitively, this means that swapping any input vectors to the differential form introduces a sign change, this is related to how we can integrate differential forms to obtain measures of length, area and volume.

## Example
Suppose that our manifold $M$ is just the euclidean plane $\mathbb{R}^2$ and suppose a 1-form would be $\omega = 3x dx - 4 y^2 dy$. Suppose also that we have a vector field $X$ defined on the plane, $X(x, y) = x^2 \frac{\partial}{\partial x} - 9 \frac{\partial}{\partial y}$.   
Then we can compute the application of the 1-form $\omega$ to the vector field $X$,

$$
\begin{align}
\omega(X) &= (3x dx - 4 y^2 dy) (x^2 \frac{\partial}{\partial x} - 9 \frac{\partial}{\partial y}) \\
\omega(X) &= 3x dx(x^2 \frac{\partial}{\partial x} - 9 \frac{\partial}{\partial y}) - 4y^2dy(x^2 \frac{\partial}{\partial x} - 9 \frac{\partial}{\partial y}) \\
\omega(X) &= 3x^3 + 36y^2
\end{align}
$$

using the property of $dx^i(\frac{\partial}{\partial x_j}) = \delta_i^j$ for coordinate components $i$ and $j$.

Essentially, now we have defined a real number to each point on the manifold $M$ corresponding to how well the vector field $X$ aligns with the 1-form $\omega$.

# Curve representation

Now, we can create a differential form representation for some closed 3-dimensional curve $\Gamma \subset \mathbb{R}^3$ using some well-known definitions in differential geometry.

Assuming we have a parameterization $\gamma : [0, 1] \to \mathbb{R}^3, \gamma(t) = (x_1(t), x_2(t), x_3(t))$ and the curve is smooth, we can define a vector field on $\Gamma$ by assigning the tangent vector of the curve to each point on $\Gamma$. The tangent vector field is therefore $T_\Gamma(p) = \gamma'(t) = \sum_{i = 1}^3 \frac{\partial x_i}{\partial t} \frac{\partial}{\partial x_i}$ for a point $p = \gamma(t)$.

Now lifting this tangent vector field $T_\Gamma$ to the ambient space of $\mathbb{R}^3$, we have a representation of the curve on $\mathbb{R}^3$. However, for any point not on the curve, we have not defined the tangent vector field. 

To define $T_\Gamma$ on the ambient space, we want the vector field to only be localized at those points which lie on the curve. Therefore, we want something like a dirac-delta "function" or rather _distribution_ $\delta$. In the spirit of the dirac distribution, we can define the tangent vector field on the ambient space by considering the sequence of vector fields $v^\epsilon_\Gamma$


$$
v^\epsilon_\Gamma (x) =
\begin{cases}
\frac{1}{\epsilon^2 \pi} T_\Gamma (\text{ClosestPoint}(x, \Gamma)), & \text{if } \text{dist}(x, \Gamma) \leq \epsilon \\
0, & \text{otherwise}
\end{cases}
$$

These vector fields are defined for all points $x \in \mathbb{R}^3$ but characterize the curve $\Gamma$ in the limit as $\epsilon \to 0$.

## Visualization of curves

To visualize these $\delta_\Gamma$ forms, we can look at some element of the sequence of vector fields, letting $\epsilon = 0.1$ for example.

Practically, we utilize Python and the community version of Manim to visualize these fields and also use them to visualize our journey going forward as well.  
In these blog posts, we only cover the relevant parts of the code and the related main ideas, the full code is available at Github https://github.com/UrjalaCoder/Minimal_surfaces.

In [1]:
%load_ext manim

The manim module is not an IPython extension.


In [ ]:
%%manim -ql -v WARNING scene
from manim import *
import numpy as np
from src.grid import Grid
from src.vector_field import VectorField
from src.utils.curve_generator import ellipse_function
from src.animation.animate_field import build_vector_field_scene
from src.utils.numerical_methods import closest_point_on_curve, numerical_tangent_function

# Instantiate the grid
grid = Grid(resolution=32)

# Instantiate the curve
curve_function = lambda t: ellipse_function(t, a = 0.4, b = 0.4, base_point=[0.5, 0.5, 0.5])

# Actual points of the curve
t_range = np.linspace(0, 1, grid.res[0])
curve = np.array([curve_function(t) for t in t_range])
epsilon = 0.1

# Visualize the curve
field = VectorField(grid=grid)
def dirac_delta_form(point: np.array):
   # Do not display the whole set of vectors
   if abs(point[2] - 0.5) > 0.2:
      return np.zeros_like(point)
   
   # Finding the closest point to the curve
   q, t_closest = closest_point_on_curve(point, curve)
   
   # Tangent vector value in the point on the curve
   if np.sum((q - point) ** 2) <= (epsilon ** 2):
      tangent = numerical_tangent_function(t_range[t_closest], curve_function, h = 0.001)
      return tangent * (1.0 / (np.pi * epsilon ** 2)) * 0.1 # Coefficient 0.1 is for visualization purposes

   return np.zeros_like(point)
field.set_vectors_from_function(dirac_delta_form)
print("building the scene...")
scene = build_vector_field_scene(grid, field, curve, max_vectors=10)


building the scene...


Manim Community v0.18.1

### Dirac-$\delta$ 2-Form Representation

Instead of describing the curve using a parametric representation alone, we adopt the **geometric measure theory** perspective, where the curve is represented as a **current** in $\mathbb{R}^3$.

The **Dirac-$\delta$ 2-form** associated with $\Gamma$ is given by:
$$
\delta_{\Gamma} \in \Omega^2(\mathbb{R}^3),
$$
which satisfies the fundamental integral identity:
$$
\int_{\mathbb{R}^3} \omega \wedge \delta_{\Gamma} = \int_{\Gamma} \omega,
$$
for any smooth test 2-form $\omega$.

This formulation naturally encodes the boundary conditions when solving the Plateau problem.

### Convergence of Integrals and Dirac-$\delta$ Representation

The Dirac-$\delta$ representation of $\Gamma$ can be understood as the limit of integrals involving smooth vector fields. Consider a sequence of smooth approximations $V_{\Gamma}^{\epsilon}$ defined in a small neighborhood around $\Gamma$, given by
$$
V_{\Gamma}^{\epsilon}(x) = \frac{1}{\epsilon^2} \sum_{p \in \Gamma} T_{\Gamma}(p) \cdot \chi_\epsilon(x - p),
$$
where $\chi_\epsilon(x)$ is a mollifier function that localizes the vector field around $\Gamma$. Then, in the limit as $\epsilon \to 0$, the integral
$$
\int_{\mathbb{R}^3} V_{\Gamma}^{\epsilon} \cdot X \, dV
$$
for any smooth vector field $X$ converges to the line integral along $\Gamma$:
$$
\int_{\Gamma} X \cdot T_{\Gamma} \, ds.
$$
This illustrates how the Dirac-$\delta$ 2-form emerges naturally as the weak limit of such integral representations.

### Musical Isomorphism and Vector Field Representation

Using the **musical isomorphism**, we can associate the Dirac-$\delta$ 2-form with a **vector field** in $\mathbb{R}^3$. Specifically,
$$
V_{\Gamma} = (\delta_{\Gamma})^{\sharp},
$$
where the **sharp operator** ($\sharp$) transforms the 2-form into a vector field.

This means that $V_{\Gamma}$ is a **vector field perpendicular to the local tangent planes** of $\delta_{\Gamma}$, encoding the circulation of the curve in a distributional sense.

---

## Surface Representation via Differential Forms

Once we have established a representation for curves, we can extend this idea to **surfaces**.

A minimal surface $\Sigma$ spanning a given boundary curve $\Gamma$ is defined as a critical point of the **area functional**:
$$
\text{Area}(\Sigma) = \int_{\Sigma} dS.
$$

### Dirac-$\delta$ 1-Form Representation

Using the differential forms framework, we define the **Dirac-$\delta$ 1-form** associated with $\Sigma$ as:
$$
\delta_{\Sigma} \in \Omega^1(\mathbb{R}^3),
$$
which satisfies the integral relation:
$$
\int_{\mathbb{R}^3} \eta \wedge \delta_{\Sigma} = \int_{\Sigma} \eta,
$$
for any smooth test 1-form $\eta$.

### Convergence of Integrals for Surface Representation

Similar to curves, the Dirac-$\delta$ 1-form representation of $\Sigma$ can be understood as the limit of integrals of smooth vector fields. Consider a sequence of smooth approximations $V_{\Sigma}^{\epsilon}$ defined in a small neighborhood around $\Sigma$ by
$$
V_{\Sigma}^{\epsilon}(x) = \frac{1}{\epsilon} \sum_{p \in \Sigma} N_{\Sigma}(p) \cdot \chi_\epsilon(x - p),
$$
where $N_{\Sigma}$ is the unit normal vector field of $\Sigma$ and $\chi_\epsilon$ is a mollifier function. Then, in the limit as $\epsilon \to 0$, the integral
$$
\int_{\mathbb{R}^3} V_{\Sigma}^{\epsilon} \cdot X \, dV
$$
for any smooth vector field $X$ converges to the flux integral over $\Sigma$:
$$
\int_{\Sigma} X \cdot N_{\Sigma} \, dS.
$$
This shows how the Dirac-$\delta$ 1-form naturally arises as the weak limit of approximating smooth fields.

### Musical Isomorphism and Surface Normal Representation

Using the musical isomorphism, we express the normal field of the minimal surface as:
$$
N_{\Sigma} = (\delta_{\Sigma})^{\sharp},
$$
where the **sharp operator** ($\sharp$) transforms the 1-form into a vector field.

Thus, the minimal surface is represented by a **normal vector field perpendicular to its tangent planes**, making it well-suited for optimization formulations.

---

## Reformulating the Problem as Convex Optimization

A key insight of Wang and Chern's approach is that **this differential form representation turns the Plateau problem into a convex optimization problem**. Instead of directly minimizing the area functional, we seek to minimize the **mass norm** of the current:
$$
\|\delta_{\Sigma}\|_{\text{mass}} = \sup_{\|\omega\|_{\max} \leq 1} \int_{\mathbb{R}^3} \omega \wedge \delta_{\Sigma}.
$$
This change of perspective **avoids the pitfalls of classical non-convex minimization** and ensures that the solution is the **true global minimum**, rather than a local minimum.

---

## Conclusion and Next Steps

In this post, we introduced the **differential form representation** of curves and surfaces, laying the groundwork for a geometric measure theory-based approach to Plateau’s problem. This formulation provides a **convex optimization perspective**, allowing us to compute minimal surfaces more reliably than traditional numerical methods.

In the next installment, we will dive deeper into the **numerical implementation** of this method, translating the mathematical formalism into practical algorithms. We will explore:

- How to efficiently **discretize differential forms** on a computational grid,
- How to **solve the convex optimization problem** using ADMM,
- How to **visualize minimal surfaces** using this approach.

By leveraging the **power of differential forms and convex optimization**, we gain a novel and elegant way to compute **true minimal surfaces**—a perspective that blends **differential geometry, numerical PDEs, and optimization** in a truly exciting way.

Stay tuned!